# Setup

In [ ]:
import os
# Sent Environment Variables
print("Environment variables set successfully!")

Environment variables set successfully!


In [2]:
%cd ..

/scratch/groups/jamesz/batu/TextualFeedback


In [3]:
import os
import numpy as np
from collections import Counter
import random

from transformers import  AutoTokenizer
from datasets import load_dataset, Dataset
# from artsco.utils import process_dataset, MODELS, apply_chat_template

from artsco.voter.voters import Voters
from artsco.voter.utils import load_persona100

# CURRENT_MODELS = ["Qwen/Qwen3-8B",  "meta-llama/Llama-3.1-8B-Instruct", "Qwen/Qwen3-14B"]
QWEN_MODEL_NAMES = ["Qwen/Qwen3-8B"] #, "Qwen/Qwen3-32B"]
LLAMA_MODEL_NAMES = ["meta-llama/Llama-3.1-8B-Instruct"] #, "meta-llama/Llama-3.1-70B-Instruct"]
OPENAI_MODEL_NAMES = ["openai/gpt-oss-20b"]
MODELS = QWEN_MODEL_NAMES + LLAMA_MODEL_NAMES 
TASKS = ["task_elections", "task_sales" , "task_sm"]
SPLITS = ["train"]
METHODS =["base", "rft", "tfb"]

/scratch/groups/jamesz/batu/miniconda3/envs/hfds-py312/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/scratch/groups/jamesz/batu/TextualFeedback/artsco/voter/voters.py:70: SyntaxWarning: invalid escape sequence '\%'
  "I know you've been looking for a tool that seamlessly integrates into your current workflow without disruption. That's why we designed our product to be plug-and-play, delivering immediate value from day one. One of our clients in your industry reduced operational costs by 23\% in just three months—without adding extra staff or hours. We're here to make your work easier, faster, and more rewarding, so your customers notice the difference immediately.",


# Prepare Second Stage Data

In [4]:
TASKS = ["task_elections", "task_sales" , "task_sm"]

results_path_root = "artsco/res"
split = "test"
datasets = {task: {model: {method: load_dataset("json", data_files=os.path.join(results_path_root,  task, model, method, f"{split}_step2.json"))['train'] for method in METHODS} for model in MODELS} for task in TASKS} 

num_voters = 20
bios = load_persona100()[:num_voters]

# voter_votes, voter_thinks = voters.get_votes_list(player_candidates)


all_candidates0 = {task: {model: {method: [] for method in METHODS} for model in MODELS} for task in TASKS} 
all_candidates1 = {task: {model: {method: [] for method in METHODS} for model in MODELS} for task in TASKS} 

TASK = ["task_elections", "task_sales" , "task_sm"]
TASK = "task_sm"
TASKS = [TASK]
for task in TASKS:
    # voters = Voters(bios=bios, task=task, model_name= "gpt-4o-mini")
    for model in MODELS:
        for method in METHODS:
            ds = datasets[task][model][method]
            cands = ds["player_candidates"]
            cands0 = [c[0] for c in cands]
            cands1 = [c[1] for c in cands]

            all_candidates0[task][model][method] = cands0
            all_candidates1[task][model][method] = cands1    

In [5]:
voters = Voters(bios=bios, task=task, model_name= "gpt-4o-mini")

METHOD_PAIRS =[ ("base", "rft"),  ("base", "tfb"),  ("rft", "tfb")]
# how superior is the second over the first
# 0.5 menas equal, 0.9 menas the second is better, 0.1 menas the first is better
# we want to see >0.5

# means =  
# stds = 
results = {}
results_std = {}
for task in TASKS:
    results[task] = {}
    results_std[task] = {}

    voters = Voters(bios=bios, task=task, model_name= "gpt-4o-mini")
    for model in MODELS:
        results[task][model] = {}
        results_std[task][model] = {}
        for pair in METHOD_PAIRS:
            player_candidates = list(zip(all_candidates0[task][model][pair[0]], all_candidates0[task][model][pair[1]]))[:]

            voter_votes, voter_thinks, voter_choices = voters.get_votes_list(player_candidates)
            mean = np.mean([np.round((Counter(v)[1] / (Counter(v)[1] + Counter(v)[0])), 2).item() for v in voter_votes])
            std = np.std([np.round((Counter(v)[1] / (Counter(v)[1] + Counter(v)[0])), 2).item() for v in voter_votes])
            
            results[task][model]["-".join(pair)] = mean
            results_std[task][model]["-".join(pair)] = std
            print(pair, "Mean:", mean, "Std:", std)

import json
json_path = f'artsco/res/{TASK}_final.json'
with open(json_path, 'w') as f:
    json.dump({"mean": results, "std": results_std}, f, indent=2)

('base', 'rft') Mean: 0.523681640625 Std: 0.21871298841988865


('base', 'tfb') Mean: 0.556962890625 Std: 0.23916410134653213


('rft', 'tfb') Mean: 0.5416406250000001 Std: 0.2526674950149096


('base', 'rft') Mean: 0.5509765625 Std: 0.31097129753111236


('base', 'tfb') Mean: 0.558349609375 Std: 0.30990502394610336


('rft', 'tfb') Mean: 0.5028320312499999 Std: 0.3178707037247991
